**Objective :**
Create the neural network model by using tensorflow so as to predict the canser servival at 3 Year'.

**Steps :**
1. Review data by checking the 'null value' in data. (In fact, I reviewed dataset on Microsoft Excel so as to see overall data first.)
2. There are 849 features(columns) in dataset and I need to predict the answer in YES/NO, so I decide to use 'Neuro Network Prediction Model'
3. Check null data & cleaning data.
4. Balancing input data.
5. Shuffle input and target data.
6. Split dataset for training, testing and validation data.
7. Export training, testing and validation data for Tensorflow format.
8. Run model.
9. Check the accuracy of model.
10. Summarize.

---

**Let start!!**


Mount Google Drive and connect colab with google drive to make sure I can load data.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Load data set.

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

path='/content/drive/My Drive/Chula UTC AI Academy /Chula UTC AI Academy - Practical Exam/Problem2/CancerRadiomicsKaggle_v2_050620.csv'

data_ground_truth = np.loadtxt(path, delimiter = ',' ,skiprows=1)
print('Size or original dataset : ' + str(data_ground_truth.shape))

Size or original dataset : (197, 852)


Preprocess the data. I split raw data to 2 groups,'input' & 'target' data

In [ ]:
# Remove record that has 'Patient Status at 3 Year' status == -1 (Unknow status.)
data_ground_truth_without_unknow_status = data_ground_truth[np.logical_and(data_ground_truth[:,9]>=0, data_ground_truth[:,9]<=1)]
print('Size or original dataset after remove record that has \'Patient Status at 3 Year\' status == unkown : ' + str(data_ground_truth_without_unknow_status.shape))

# Delele colum 'Patient ID', 'Time to Event', 'Patient Status at 3 Year'
unscaled_input_data_ground_truth = np.delete(data_ground_truth_without_unknow_status, [0,7,9], axis=1)
print('Size of input data after delete 3 columns : '+ str(unscaled_input_data_ground_truth.shape))

# All target data
targets_data_ground_truth = data_ground_truth_without_unknow_status[:, 9]
print('Size of target data : ' + str(targets_data_ground_truth.shape))

Size or original dataset after remove record that has 'Patient Status at 3 Year' status == unkown : (195, 852)
Size of input data after delete 3 columns : (195, 849)
Size of target data : (195,)


Balancing input & target data.

In [ ]:
# Check Number of death patient in target data.
number_death_targets = int(np.sum(targets_data_ground_truth))
print("Number of death targets : " + str(number_death_targets))
number_zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_data_ground_truth.shape[0]):
  # Check only target == 0 (Not Death)
  if targets_data_ground_truth[i] == 0:
    number_zero_targets_counter += 1
    # If number of 0 target is morethan number of 1.
    if number_zero_targets_counter > number_death_targets:
        # Keep id that need to remove into indices_to_remove array
        indices_to_remove.append(i)
        #print(targets_data_ground_truth[i])

# Delete record in 'unscaled_input_data_ground_truth' and 'targets_data_ground_truth' that has the same id in indices_to_remove array.
unscaled_input_equal_priors = np.delete(unscaled_input_data_ground_truth, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_data_ground_truth, indices_to_remove, axis=0)

print("Size of unscale input data : " + str(unscaled_input_equal_priors.shape))
print("Size of target data : " + str(targets_equal_priors.shape))

Number of death targets : 31
Size of unscale input data : (62, 849)
Size of target data : (62,)


Scale input data by implementing scale feature.

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_input_equal_priors)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Shuffle input and target data.

In [ ]:
# Create shuffle index
shuffled_indices = np.arange(scaled_inputs.shape[0])
# Random shuffle
np.random.shuffle(shuffled_indices)

# Apply index into input&target array.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

print(shuffled_inputs)
print(shuffled_targets)

[[-4.38529010e-01  1.42045487e+00  3.71811446e-01 ... -3.14473460e-01
  -7.04995948e-01  7.76010159e-01]
 [-4.38529010e-01 -8.48894240e-01  1.29390383e+00 ...  7.29483232e-02
  -3.14302443e-01 -4.20828115e-01]
 [-4.38529010e-01 -2.37915633e-01 -5.50280940e-01 ... -2.79765531e-01
  -7.05481516e-01 -1.12919295e-01]
 ...
 [-4.38529010e-01  2.39323418e-02 -5.50280940e-01 ...  9.18079957e-02
  -1.59142102e-02 -8.83393773e-01]
 [-4.38529010e-01  5.47628291e-01 -1.47237333e+00 ...  6.90576675e-02
  -6.95271603e-01 -1.41654643e-01]
 [-4.38529010e-01  2.39323418e-02  1.29390383e+00 ... -6.57575825e-01
  -6.57193403e-04  9.49592036e-03]]
[1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1.]


Devide dataset into train, validation and test for preventing the overfitting.
![alt text](https://www.aoddy.com/wp-content/uploads/2020/05/Split_data.png)

In [ ]:
# Check number of input data
samples_count = shuffled_inputs.shape[0]
print("Total sample data : " + str(samples_count))

# I split data to 
#   80% for training
#   10% for validating
#   10% for testing

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
testing_samples_count = samples_count - train_samples_count - validation_samples_count

print("Training sample number : " + str(train_samples_count))
print("Validating sample number : " + str(validation_samples_count))
print("Testing sample number : " + str(testing_samples_count))

# Split shuffled input & target data
train_inputs = shuffled_inputs[: train_samples_count]
validation_inputs = shuffled_inputs[train_samples_count : train_samples_count + validation_samples_count]
test_inputs = shuffled_inputs[train_samples_count + validation_samples_count :]

train_targets = shuffled_targets[: train_samples_count]
validation_targets = shuffled_targets[train_samples_count : train_samples_count + validation_samples_count]
test_targets = shuffled_targets[train_samples_count + validation_samples_count : ]

print("Double check size of input data : " + str(train_inputs.shape[0] + validation_inputs.shape[0] + test_inputs.shape[0]))
print("Double check size of target data : " + str(train_targets.shape[0] + validation_targets.shape[0] + test_targets.shape[0]))


Total sample data : 62
Training sample number : 49
Validating sample number : 6
Testing sample number : 7
Double check size of input data : 62
Double check size of target data : 62


Save training, validation and testing data for tensorfow format (.npz)

In [ ]:
np.savez('CancerRadiomics_data_train', inputs=train_inputs, targets=train_targets)
np.savez('CancerRadiomics_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('CancerRadiomics_data_test', inputs=test_inputs, targets=test_targets)

Load training, validation and testing data from files.

In [ ]:
npz = np.load('CancerRadiomics_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('CancerRadiomics_data_validation.npz')
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('CancerRadiomics_data_test.npz')
test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

Run model. - The input size is number of features and output size is 2 because the answers is YES or NO.

In [ ]:
input_size = 849
output_size = 2
hidden_layer_size = 566

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks=[early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose=2)

Epoch 1/100
1/1 - 0s - loss: 1.0142 - accuracy: 0.4898 - val_loss: 2.4880 - val_accuracy: 0.3333
Epoch 2/100
1/1 - 0s - loss: 1.5885 - accuracy: 0.5102 - val_loss: 0.5613 - val_accuracy: 0.5000
Epoch 3/100
1/1 - 0s - loss: 0.5353 - accuracy: 0.7959 - val_loss: 0.0754 - val_accuracy: 1.0000
Epoch 4/100
1/1 - 0s - loss: 0.3326 - accuracy: 0.8776 - val_loss: 0.0460 - val_accuracy: 1.0000
Epoch 5/100
1/1 - 0s - loss: 0.3720 - accuracy: 0.7959 - val_loss: 0.0651 - val_accuracy: 1.0000
Epoch 6/100
1/1 - 0s - loss: 0.2050 - accuracy: 0.8776 - val_loss: 0.1802 - val_accuracy: 1.0000


Summarize 
The accuracy of this model is 85% but the percent loss is quit high. So if we have more ground truth data, the number of accuracy should be increase and percent loss should be reduce. 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

1/1 [==============================] - 0s 1ms/step - loss: 0.1610 - accuracy: 0.8571


In [ ]:
print('\n# Generate predictions for 3 samples')
predictions = model.predict(test_inputs[:3])
print('predictions shape:', predictions.shape)
print(predictions)


# Generate predictions for 3 samples
predictions shape: (3, 2)
[[9.7574371e-01 2.4256352e-02]
 [6.4135375e-03 9.9358648e-01]
 [9.9986994e-01 1.3009210e-04]]
